In [1]:
import napari_workflows
from napari_workflows._io_yaml_v1 import load_workflow
import napari
import napari_pyclesperanto_assistant
from napari_pyclesperanto_assistant._workflow_io_utility import old_wf_names_to_new_mapping, wf_steps_with_root_as_input, initialise_root_functions
from skimage.io import imread

filename = 'test_workflow_multiple_nodes.yaml'
workflow = load_workflow(filename)
print(workflow)

Workflow:
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x000001502E24A290>, 'blobs_messed_up', 4.0)
Result of Gaussian (scikit-image, nsbatwm) [1] <- (<function gaussian_blur at 0x000001502E24A290>, 'blobs', 3.0)
Result of Maximum (scipy, nsbatwm) <- (<function maximum_filter at 0x000001502E24AA70>, 'Result of Gaussian (scikit-image, nsbatwm)', 24.0)
Result of Sum images (numpy, nsbatwm) <- (<function sum_images at 0x000001502E24B760>, 'Result of Gaussian (scikit-image, nsbatwm) [1]', 'Result of Maximum (scipy, nsbatwm)', 1.0, 1)



In [ ]:
desktop = 'C:/Users/ryans/Desktop/'
image1 = imread(desktop + 'blobs.tif')
image2 = imread(desktop + 'blobs_messed_up.tiff')

viewer = napari.Viewer()
viewer.add_image(image1)
viewer.add_image(image2)

In [3]:
root_functions = wf_steps_with_root_as_input(workflow=workflow)
initialise_root_functions(workflow=workflow, 
                          viewer=viewer)

In [ ]:
from napari_pyclesperanto_assistant._workflow_io_utility import load_remaining_workflow
load_remaining_workflow(workflow=workflow,viewer=viewer)